**3m Term SOFR Cap**

**SOFRスワップ**
**([settleDays = 0 でのエラー回避法](https://stackoverflow.com/questions/37159215/jpylibor-fixing-during-japanese-holiday-negative-time-error))**
- curveOBJ.discount(date) のdateがカーブsettleDTよりも過去の場合 マイナスの日付でエラー
- PiecewiseLogLinearDiscount(0, ... ) で回避可能  
<img src='SOFR-Curve-Sep26,2023.png' width='600'>

In [1]:
# crvDATA = [ ('depo','1d',5.31),    ('swap','1w',5.30978),  ('swap','2w',5.31252),
#             ('swap','3w',5.31526), ('swap','1m',5.31958),  ('swap','2m',5.35225),
#             ('swap','3m',5.3826),  ('swap','4m',5.413),    ('swap','5m',5.43675),
#             ('swap','6m',5.455),   ('swap','7m',5.46963),  ('swap','8m',5.47425),
#             ('swap','9m',5.47415), ('swap','10m',5.47165), ('swap','11m',5.46235),
#             ('swap','12m',5.45038),('swap','18m',5.22235), ('swap','2y',5.00513),
#             ('swap','3y',4.67081), ('swap','4y',4.48275),  ('swap','5y',4.38035),
#             ('swap','6y',4.32195), ('swap','7y',4.28295),  ('swap','8y',4.259),
#             ('swap','9y',4.24565), ('swap','10y',4.23835), ('swap','12y',4.23685),
#             ('swap','15y',4.2395), ('swap','20y',4.195),   ('swap','25y',4.08925),
#             ('swap','30y',3.98355),('swap','40y',3.7615),  ('swap','50y',3.54745)]    

In [2]:
import QuantLib as ql ; import pandas as pd 
import myUtil as mu   ; from myABBR import * ; from scipy.stats import norm
tradeDT = mu.jDT(2023,9,26)  ; mu.setEvDT(tradeDT)

def makeSofrCurve(crvDATA):
  '''makeSofrCurve(crvDATA)->[sofrIX,sfCrvOBJ,sfCrvHDL,sfParRATE]'''      
  # 1.指標金利オブジェクトと初期値設定
  sfCrvHDL = ql.RelinkableYieldTermStructureHandle()  
  sofrIX = ql.Sofr(sfCrvHDL)
  # 2. HelperとSOFRカーブオブジェクト
  cHelper, sfParRATE = [], []
  for knd, tnr, rt in crvDATA:
      if knd == 'depo':
          if ql.Period(tnr).length() == 1:
              cHelper.append(ql.DepositRateHelper(mu.sqHDL(rt/100),sofrIX)) 
      if knd == 'swap': cHelper.append(
          ql.OISRateHelper(Tp2, ql.Period(tnr),mu.sqHDL(rt/100),sofrIX))
      sfParRATE.append(rt/100)                             # パーレート用リスト
  sfCrvOBJ = ql.PiecewiseLogLinearDiscount(Tp0, calUSs, cHelper, dcA360)
  sfCrvHDL.linkTo(sfCrvOBJ) ; sfCrvOBJ.enableExtrapolation()
  return [sofrIX, sfCrvOBJ, sfCrvHDL, sfParRATE]      # 4つのオブジェクトを戻す

In [3]:
crvDATA = [ ('depo','1d',5.31),    ('swap','1w',5.30978),  ('swap','2w',5.31252),
            ('swap','3w',5.31526), ('swap','1m',5.31958),  ('swap','2m',5.35225),
            ('swap','3m',5.3826),  ('swap','4m',5.413),    ('swap','5m',5.43675),
            ('swap','6m',5.455),   ('swap','7m',5.46963),  ('swap','8m',5.47425),
            ('swap','9m',5.47415), ('swap','10m',5.47165), ('swap','11m',5.46235),
            ('swap','12m',5.45038),('swap','18m',5.22235), ('swap','2y',5.00513),
            ('swap','3y',4.67081), ('swap','4y',4.48275),  ('swap','5y',4.38035),
            ('swap','6y',4.32195), ('swap','7y',4.28295),  ('swap','8y',4.259),
            ('swap','9y',4.24565), ('swap','10y',4.23835), ('swap','12y',4.23685),
            ('swap','15y',4.2395), ('swap','20y',4.195),   ('swap','25y',4.08925),
            ('swap','30y',3.98355),('swap','40y',3.7615),  ('swap','50y',3.54745)] 

In [4]:
# SOFRカーブ作成
sofrIX, sfCrvOBJ, sfCrvHDL, sfParRATE = makeSofrCurve(crvDATA)
nodesDF = pd.DataFrame([(dt.ISO(),df) for dt,df in sfCrvOBJ.nodes()],
                                                      columns=['','DF'])
print("決済日(reference): ", sfCrvOBJ.referenceDate().ISO())
nodesDF[1:5].style.format({'DF':'{:.9f}' })

決済日(reference):  2023-09-26


,,DF
1,2023-09-27,0.999852522
2,2023-10-05,0.998674048
3,2023-10-12,0.997644024
4,2023-10-19,0.996615063


**sofrカーブからのTerm Sofrカーブ作成**

<img src='TsfrCrvData-Sep26,2023.png' width='600'>

In [5]:
# Term SOFR rate and Basis curve ( = all zero)
TsfRT3m  = 5.38558
TsfCrvBS = [('6m',0), ('9m',0), ('12m',0),('18m',0),('2y',0), ('3y',0), ('4y',0), 
            ('5y',0), ('6y',0), ('7y',0), ('8y',0), ('9y',0), ('10y',0),('12y',0),
            ('15y',0),('20y',0),('25y',0),('30y',0),('40y',0),('50y',0)          ]

# 1.TermSOFR指標金利オブジェクト
TsfCrvHDL = ql.RelinkableYieldTermStructureHandle()  
TsfIX = ql.IborIndex('TermSofr', pdFreqQ, Tp2, usdFX, calUSs, mFLLW,
                                                      EoMf, dcA360, TsfCrvHDL)
# 2. Basis helperでのTermSOFRカーブオブジェクト
cHelper = [ql.DepositRateHelper(mu.sqHDL(TsfRT3m/100),TsfIX)]
for tnr, rt in TsfCrvBS:
    cHelper.append(
            ql.OvernightIborBasisSwapRateHelper(mu.sqHDL(rt/100),
            ql.Period(tnr), Tp2, calUSs, mFLLW, EoMf, sofrIX, TsfIX, sfCrvHDL))
TsfCrvOBJ = ql.PiecewiseLogLinearDiscount(Tp2, calUSs, cHelper, dcA360)
TsfCrvHDL.linkTo(TsfCrvOBJ) ; TsfCrvOBJ.enableExtrapolation()
# checking
print("決済日(reference): ", TsfCrvOBJ.referenceDate().ISO())
[(dt.ISO(), df) for dt, df in TsfCrvOBJ.nodes()][:5]

決済日(reference):  2023-09-28


[('2023-09-28', 1.0),
 ('2023-12-28', 0.9865692901876598),
 ('2024-03-28', 0.9731621804931488),
 ('2024-06-28', 0.9600022193693677),
 ('2024-09-30', 0.9472254399920353)]

**アニュイティ、フォワードレートの計算**  
- Flat version


<img src='Tsf3m-Cap5p-1y-Main-Sep26,2023.png' width='500'>

<img src='Tsf3m-Cap5p-1y-CF-Sep26,2023.png' width='800'>

In [6]:
# cap条件とSchedule
effDT,                 matDT,         capSTK,   ntlAMT,     volRT     = \
mu.jDT(2023,9,28), mu.jDT(2024,9,28),  0.05,  10_000_000,  0.88/100

capSCD  = ql.Schedule(effDT,matDT,pdFreqQ,calUSs,mFLLW,mFLLW,dgRULEb,EoMf)
print('Schedule: ', [dd.ISO() for dd in list(capSCD)]) # checking

# プライシング
capENG = ql.BachelierCapFloorEngine(sfCrvHDL, mu.sqHDL(volRT))   
capLeg = ql.IborLeg([ntlAMT], capSCD, TsfIX, dcA360)
capOBJ = ql.Cap(capLeg, [capSTK]) ; capOBJ.setPricingEngine(capENG)
print(f'capNPV  :   {capOBJ.NPV():,.2f}')

Schedule:  ['2023-09-28', '2023-12-28', '2024-03-28', '2024-06-28', '2024-09-30']
capNPV  :   42,851.56


In [7]:
# caplet明細
dfLET = pd.DataFrame(dict(
            stdDEV = (capOBJ.optionletsStdDev()),    #stdDev=σ√T
            atmFWD = (capOBJ.optionletsAtmForward()),
            letDF  = (capOBJ.optionletsDiscountFactor()), 
            letNPV = (capOBJ.optionletsPrice())          ))
dfLET.style.format({'atmFWD':'{:.6%}', 'letNPV':'{:,.2f}'})

,stdDEV,atmFWD,letDF,letNPV
0,0.000000,5.385580%,0.986286,"9,612.94"
1,0.004394,5.450183%,0.972875,"11,929.92"
2,0.006214,5.364102%,0.959719,"11,559.98"
3,0.007625,5.165861%,0.946946,"9,748.72"


In [8]:
# caplet日付関連
dfCap = pd.DataFrame({
    'matYR':      dcA365.yearFraction(tradeDT, cpn.fixingDate()), #maturity year
    'fixingDate': cpn.fixingDate().ISO(),
    'accruStart': cpn.accrualStartDate().ISO(),
    'accruEnd':   cpn.accrualEndDate().ISO(),
    'payDate':    cpn.date().ISO(),
    'days':       dcA360.dayCount(cpn.accrualStartDate(),cpn.accrualEndDate()),
    'TsfDF':      TsfCrvOBJ.discount(cpn.date())            #calc forward rate 
    } for cpn in map(ql.as_floating_rate_coupon, capLeg)) 

dfCap = pd.concat([dfCap, dfLET], axis=1)
dfCap.style.format({'matYR':'{:.4f}','TsfDF':'{:.8f}','atmFWD':'{:.6%}', 'letNPV':'{:,.2f}'})

,matYR,fixingDate,accruStart,accruEnd,payDate,days,TsfDF,stdDEV,atmFWD,letDF,letNPV
0,0.0000,2023-09-26,2023-09-28,2023-12-28,2023-12-28,91,0.98656929,0.000000,5.385580%,0.986286,"9,612.94"
1,0.2493,2023-12-26,2023-12-28,2024-03-28,2024-03-28,91,0.97316218,0.004394,5.450183%,0.972875,"11,929.92"
2,0.4986,2024-03-26,2024-03-28,2024-06-28,2024-06-28,92,0.96000222,0.006214,5.364102%,0.959719,"11,559.98"
3,0.7507,2024-06-26,2024-06-28,2024-09-30,2024-09-30,94,0.94722544,0.007625,5.165861%,0.946946,"9,748.72"


In [9]:
# caplet3の手計算 (PC:call=1 put=-1, TT:満期年, SD:stdDev=σ√T)
PC,        FF,         XX,         TT,              SD        =\
1,  dfCap.atmFWD[3], capSTK, dfCap.matYR[3], dfCap.stdDEV[3]

# オプション計算
d1     = PC*(FF-XX)/SD
optPRC = SD*(d1*norm.cdf(d1) + norm.pdf(d1))
optAMT = dfCap.letDF[3] * optPRC * dfCap.days[3]/360 * ntlAMT
print(f'optPRC:{optPRC:.6%}, 受渡金額:{optAMT:,.2f}')

optPRC:0.394273%, 受渡金額:9,748.72


In [10]:
# (参考)フォワードレート手計算
import numpy as np
DFs = np.array([TsfCrvOBJ.discount(dd) for dd in capSCD]) 
diffDF = np.diff(DFs)*(-1)      ; print('diffDF  :',diffDF)
capTNR = np.diff(np.array([dcA360.yearFraction(effDT, dd) for dd in capSCD]))
fwdRT = diffDF/(capTNR*DFs[1:]) ; print('fwdRT:',fwdRT)

diffDF  : [0.01343071 0.01340711 0.01315996 0.01277678]
fwdRT: [0.0538558  0.05450183 0.05364102 0.05165861]
